<a href="https://colab.research.google.com/github/aditya-sonone/FInal_Year_Project/blob/main/FYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!unzip /content/FYP.zip
%cd /content/content

Archive:  /content/FYP.zip
  inflating: content/fcc4c8a4-2177-4a08-a785-063497dfd121___RS_HL 5844_flipTB.JPG  
  inflating: content/keras_model.h5  
  inflating: content/labels.txt      
   creating: content/templates/
  inflating: content/templates/result.html  
  inflating: content/templates/index.html  
/content/content


In [28]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("/content/content/keras_model.h5", compile=False)

# Load the labels
class_names = open("/content/content/labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("/content/content/fcc4c8a4-2177-4a08-a785-063497dfd121___RS_HL 5844_flipTB.JPG").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)

1/1 [==============================] - 1s 1s/step
Class: Apple_Healthy
Confidence Score: 0.9999547


In [29]:
!pip install pyngrok
from flask import Flask
from pyngrok import ngrok
port_no=5000

In [30]:
import os
import sys
from flask import Flask, render_template, request
from keras.models import load_model
from PIL import Image, ImageOps, ImageFile
import numpy as np
from werkzeug.utils import secure_filename
from io import BytesIO
import base64
import io

from pyngrok import ngrok

app = Flask(__name__)
model = load_model("keras_model.h5", compile=False)

class_names = open("labels.txt", "r").readlines()

fertilizer_contents = {
    "Apple_Apple_scab": ["Nitrogen", "Phosphorus", "Potassium"],
    "Apple_Black_rot": ["Nitrogen", "Calcium", "Magnesium"],
    "Apple_Cedar_apple_rust": ["Sulfur", "Copper", "Zinc"],
    "Apple_Healthy": ["No fertilizer required"],
    "Corn_Cercospora_leaf_spot Gray_leaf_spot": ["Nitrogen", "Potassium", "Magnesium"],
    "Corn_Common_rust_": ["Nitrogen", "Sulfur", "Manganese"],
    "Corn_Healthy": ["No fertilizer required"],
    "Corn_Northern_Leaf_Blight": ["Nitrogen", "Potassium", "Zinc"],
    "Potato_Early_blight": ["Nitrogen", "Phosphorus", "Potassium"],
    "Potato_Healthy": ["No fertilizer required"],
    "Potato_Late_blight": ["Nitrogen", "Phosphorus", "Potassium"],
    "Tomato_Bacterial_spot": ["Nitrogen", "Calcium", "Magnesium"],
    "Tomato_Early_blight": ["Nitrogen", "Phosphorus", "Potassium"],
    "Tomato_Healthy": ["No fertilizer required"],
    "Tomato_Late_blight": ["Nitrogen", "Phosphorus", "Potassium"]
}


data = {
    "Apple_Apple_scab": {
        "fertilizers": ["Bordeaux Mixture", "Sulfur-based fungicides"],
        "Usage": "Apply as a foliar spray during dormant and growing",
        "Working": "Controls fungal growth, prevents infection of apple scab pathogens",
        "Effect": "Promotes healthy leaf and fruit development, reduces scab incidence"
    },
    "Apple_Black_rot": {
        "fertilizers": ["Bordeaux Mixture", "Copper oxychloride"],
        "Usage": "Apply as a foliar spray during dormant and growing",
        "Working": "Controls fungal growth, prevents infection of black rot pathogens",
        "Effect": "Promotes healthy fruit development, reduces rot incidence"
    },
    "Apple_Cedar_apple_rust": {
        "fertilizers": ["Bordeaux Mixture", "Sulfur-based fungicides"],
        "Usage": "Apply as a foliar spray during dormant and growing",
        "Working": "Controls fungal growth, prevents infection of cedar apple rust fungi",
        "Effect": "Promotes healthy leaf and fruit development, reduces rust incidence"
    },
    "Apple_Healthy": {
        "fertilizers": ["No fertilizer required"],
        "Usage": "No application required",
        "Working": "No fungal infection present",
        "Effect": "Maintains existing health"
    },
    "Corn_Cercospora_leaf_spot Gray_leaf_spot": {
        "fertilizers": ["Trichoderma-based biofungicides"],
        "Usage": "Apply as a foliar spray",
        "Working": "Suppresses fungal growth, enhances plant's natural defense mechanisms",
        "Effect": "Promotes healthy leaf development, reduces leaf spot incidence"
    },
    "Corn_Common_rust_": {
        "fertilizers": ["Copper oxychloride-based fungicides"],
        "Usage": "Apply as a foliar spray during growing season",
        "Working": "Controls fungal growth, prevents infection of common rust pathogens",
        "Effect": "Promotes healthy leaf development, reduces rust incidence"
    },
    "Corn_Healthy": {
        "fertilizers": ["No fertilizer required"],
        "Usage": "No application required",
        "Working": "No fungal infection present",
        "Effect": "Maintains existing health"
    },
    "Corn_Northern_Leaf_Blight": {
        "fertilizers": ["Trichoderma-based biofungicides"],
        "Usage": "Apply as a foliar spray",
        "Working": "Suppresses fungal growth, enhances plant's natural defense mechanisms",
        "Effect": "Promotes healthy leaf development, reduces blight incidence"
    },
    "Potato_Early_blight": {
        "fertilizers": ["Bordeaux Mixture", "Copper oxychloride"],
        "Usage": "Apply as a foliar spray during growing season",
        "Working": "Controls fungal growth, prevents infection of early blight pathogens",
        "Effect": "Promotes healthy foliage, reduces blight incidence"
    },
    "Potato_Healthy": {
        "fertilizers": ["No fertilizer required"],
        "Usage": "No application required",
        "Working": "No fungal infection present",
        "Effect": "Maintains existing health"
    },
    "Potato_Late_blight": {
        "fertilizers": ["Bordeaux Mixture", "Mancozeb-based fungicides"],
        "Usage": "Apply as a foliar spray during growing season",
        "Working": "Controls fungal growth, prevents infection of late blight pathogens",
        "Effect": "Promotes healthy foliage, reduces blight incidence"
    },
    "Tomato_Bacterial_spot": {
        "fertilizers": ["Copper oxychloride-based bactericides"],
        "Usage": "Apply as a foliar spray during growing season",
        "Working": "Controls bacterial growth, prevents infection of bacterial spot pathogens",
        "Effect": "Promotes healthy foliage, reduces spot incidence"
    },
    "Tomato_Early_blight": {
        "fertilizers": ["Copper oxychloride-based fungicides"],
        "Usage": "Apply as a foliar spray during growing season",
        "Working": "Controls fungal growth, prevents infection of early blight pathogens",
        "Effect": "Promotes healthy foliage, reduces blight incidence"
    },
    "Tomato_Healthy": {
        "fertilizers": ["No fertilizer required"],
        "Usage": "No application required",
        "Working": "No fungal infection present",
        "Effect": "Maintains existing health"
    },
    "Tomato_Late_blight":{
        "fertilizers": ["Bordeaux Mixture"],
        "Usage": "Apply as a foliar spray during growing season",
        "Working": "Controls fungal growth, prevents infection of late blight pathogens",
        "Effect": "Promotes healthy foliage, reduces blight incidence"
    }
}


def preprocess_image(image):
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array
    return data

# Set the ngrok authtoken
ngrok.set_auth_token("2fE5fUkgIgLzY8HyybnPnQtsk1j_5ZTby6XSdqhXWCktTquwD")
public_url=ngrok.connect(port_no).public_url

# Home page
@app.route('/')
def home():
    return render_template('index.html')

# Prediction page
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return "No file part"

    file = request.files['file']
    if file.filename == '':
        return "No selected file"

    if file:
        image = Image.open(file).convert("RGB")
        processed_image = preprocess_image(image)

        # Predicts the model
        prediction = model.predict(processed_image)
        index = np.argmax(prediction)
        # class_name = class_names[index][2:]
        class_name = class_names[index][2:].lstrip().rstrip()
        confidence_score = prediction[0][index]

        return render_template('result.html', class_name=class_name, confidence_score=confidence_score, fertilizers=fertilizer_contents, data=data)


# Run the Flask app
print(f"Click here to access global link: {public_url}")
app.run(port=port_no)

Click here to access global link: https://cbf5-34-16-166-116.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Apr/2024 23:15:33] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2024 23:15:35] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 1s 732ms/step


INFO:werkzeug:127.0.0.1 - - [19/Apr/2024 23:15:44] "POST /predict HTTP/1.1" 200 -


In [32]:
ngrok.kill()